In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
import numpy as np
from tensorflow.keras.layers import LSTM
import pickle

# Load training data from text file
def load_training_data(file_path):
    texts = []
    labels = []
    label_map = {"Obtainable": 0, "Operable": 1, "Update": 2}  # Define label mapping
    with open(file_path, 'r') as file:
        for line in file:
            text, label = line.strip().split(',')
            texts.append(text)
            labels.append(label_map[label])  # Map label string to integer
    return texts, labels

# Sample dataset (for testing)
texts, labels = load_training_data('training_data.txt')

# Parameters
vocab_size = 5000
embedding_dim = 100
max_length = 20
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

# Tokenization and padding
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Preparing labels
labels = np.array(labels)

# Building the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
model.fit(padded_sequences, labels, epochs=10, verbose=2)

# Save the tokenizer for later use
tokenizer_path = "tokenizer.pickle"
with open(tokenizer_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the model in the native Keras format
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Function to classify new requests
def classify_request(request):
    seq = tokenizer.texts_to_sequences([request])
    padded = pad_sequences(seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    prediction = model.predict(padded)
    return np.argmax(prediction, axis=1)[0]

# Example usage
new_request = "I want to sleep"
request_type = classify_request(new_request)
request_types = ["Obtainable", "Operable", "Update"]
print(f"Request type: {request_types[request_type]}")

model.export('raw-data')
model.save("raw-data/SaveState.keras")

Epoch 1/10
2/2 - 4s - 2s/step - accuracy: 0.4035 - loss: 1.0988
Epoch 2/10
2/2 - 0s - 72ms/step - accuracy: 0.2632 - loss: 1.1003
Epoch 3/10
2/2 - 0s - 126ms/step - accuracy: 0.4386 - loss: 1.0969
Epoch 4/10
2/2 - 0s - 41ms/step - accuracy: 0.3509 - loss: 1.0962
Epoch 5/10
2/2 - 0s - 37ms/step - accuracy: 0.3684 - loss: 1.0952
Epoch 6/10
2/2 - 0s - 73ms/step - accuracy: 0.4386 - loss: 1.0890
Epoch 7/10
2/2 - 0s - 36ms/step - accuracy: 0.4211 - loss: 1.0816
Epoch 8/10
2/2 - 0s - 81ms/step - accuracy: 0.5614 - loss: 1.0613
Epoch 9/10
2/2 - 0s - 84ms/step - accuracy: 0.8947 - loss: 1.0200
Epoch 10/10
2/2 - 0s - 66ms/step - accuracy: 0.8070 - loss: 0.9465


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Request type: Obtainable
INFO:tensorflow:Assets written to: raw-data\assets


INFO:tensorflow:Assets written to: raw-data\assets


Saved artifact at 'raw-data'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 20), dtype=tf.float32, name='keras_tensor_75')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  1863679229584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863679228624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863679228816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863547516496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863547506896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863547516112: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [12]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

# Load the compiled Keras model
model = load_model('raw-data/SaveState.keras')

# Export the model to the SavedModel format
# model.save('tf_model')

# Convert the SavedModel to TFLite format with experimental flags
converter = tf.lite.TFLiteConverter.from_saved_model('raw-data')
converter.experimental_enable_resource_variables = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False

# Perform the conversion
tflite_model = converter.convert()

# Save the TFLite model
with open('1stNeuralLink.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite built, converted and saved successfully!")


h:\DevKit\projects\neuralnet\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


TFLite built, converted and saved successfully!
